In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 8
tree_depth = 12
batch_size = 512
device = 'cuda'
train_data_path = r'/mnt/qnap/ekosman/mitbih_train.csv'
test_data_path = r'/mnt/qnap/ekosman/mitbih_test.csv'

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [7]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.476043224334717 | KNN Loss: 5.769813060760498 | CLS Loss: 1.7062300443649292
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 4.464776992797852 | KNN Loss: 3.2373194694519043 | CLS Loss: 1.2274576425552368
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 3.391493320465088 | KNN Loss: 2.6543593406677246 | CLS Loss: 0.7371340990066528
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 3.1886754035949707 | KNN Loss: 2.561807632446289 | CLS Loss: 0.6268676519393921
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 3.188472032546997 | KNN Loss: 2.5338451862335205 | CLS Loss: 0.6546267867088318
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 3.1445138454437256 | KNN Loss: 2.6530253887176514 | CLS Loss: 0.49148836731910706
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 3.0928361415863037 | KNN Loss: 2.5160574913024902 | CLS Loss: 0.5767786502838135
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 2.983839511871338 | KNN Loss: 2.521601200103759

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 2.598839044570923 | KNN Loss: 2.4779484272003174 | CLS Loss: 0.12089067697525024
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 2.6051814556121826 | KNN Loss: 2.512763023376465 | CLS Loss: 0.09241849184036255
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 2.6455912590026855 | KNN Loss: 2.53206729888916 | CLS Loss: 0.11352407932281494
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 2.606755018234253 | KNN Loss: 2.515778064727783 | CLS Loss: 0.09097685664892197
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 2.613258123397827 | KNN Loss: 2.504054546356201 | CLS Loss: 0.10920365154743195
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 2.590320587158203 | KNN Loss: 2.4629762172698975 | CLS Loss: 0.1273442655801773
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 2.6003620624542236 | KNN Loss: 2.474505662918091 | CLS Loss: 0.1258564591407776
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 2.6393747329711914 | KNN Loss: 2.5004622

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 2.5393362045288086 | KNN Loss: 2.4390337467193604 | CLS Loss: 0.10030245035886765
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 2.56878662109375 | KNN Loss: 2.441195011138916 | CLS Loss: 0.12759165465831757
Epoch: 007, Loss: 2.5576, Train: 0.9771, Valid: 0.9739, Best: 0.9739
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 2.5666160583496094 | KNN Loss: 2.468266725540161 | CLS Loss: 0.09834936261177063
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 2.557650327682495 | KNN Loss: 2.459782123565674 | CLS Loss: 0.09786819666624069
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 2.5491745471954346 | KNN Loss: 2.4246954917907715 | CLS Loss: 0.12447910010814667
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 2.5385468006134033 | KNN Loss: 2.462998867034912 | CLS Loss: 0.07554784417152405
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 2.5647027492523193 | KNN Loss: 2.4557669162750244 | CLS Loss: 0.10893575102090836
Epoch 8 / 200 | iter

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 2.4872567653656006 | KNN Loss: 2.4295566082000732 | CLS Loss: 0.05770007520914078
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 2.546365261077881 | KNN Loss: 2.4590630531311035 | CLS Loss: 0.0873023197054863
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 2.5020530223846436 | KNN Loss: 2.4328184127807617 | CLS Loss: 0.0692346841096878
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 2.5381650924682617 | KNN Loss: 2.461623191833496 | CLS Loss: 0.07654178887605667
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 2.5211596488952637 | KNN Loss: 2.43990159034729 | CLS Loss: 0.08125804364681244
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 2.533362627029419 | KNN Loss: 2.455315589904785 | CLS Loss: 0.07804711163043976
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 2.4923484325408936 | KNN Loss: 2.420955181121826 | CLS Loss: 0.07139333337545395
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 2.4960505962371826 | KNN Loss: 2

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 2.499396562576294 | KNN Loss: 2.453481435775757 | CLS Loss: 0.04591507464647293
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 2.462001323699951 | KNN Loss: 2.413766384124756 | CLS Loss: 0.04823485016822815
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 2.498473882675171 | KNN Loss: 2.407113552093506 | CLS Loss: 0.09136022627353668
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 2.4779276847839355 | KNN Loss: 2.412907123565674 | CLS Loss: 0.06502047181129456
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 2.4980461597442627 | KNN Loss: 2.4089159965515137 | CLS Loss: 0.08913024514913559
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 2.499948263168335 | KNN Loss: 2.43463397026062 | CLS Loss: 0.06531422585248947
Epoch: 014, Loss: 2.4911, Train: 0.9813, Valid: 0.9778, Best: 0.9798
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 2.5111746788024902 | KNN Loss: 2.3962152004241943 | CLS Loss: 0.11495952308177948
Epoch 15 / 2

...

Epoch 185 / 200 | iteration 120 / 171 | Total Loss: 2.410003662109375 | KNN Loss: 2.4045491218566895 | CLS Loss: 0.005454644560813904
Epoch 185 / 200 | iteration 130 / 171 | Total Loss: 2.4125468730926514 | KNN Loss: 2.4065325260162354 | CLS Loss: 0.006014366168528795
Epoch 185 / 200 | iteration 140 / 171 | Total Loss: 2.4414896965026855 | KNN Loss: 2.4342782497406006 | CLS Loss: 0.007211505901068449
Epoch 185 / 200 | iteration 150 / 171 | Total Loss: 2.402709484100342 | KNN Loss: 2.4011120796203613 | CLS Loss: 0.0015975134447216988
Epoch 185 / 200 | iteration 160 / 171 | Total Loss: 2.429591417312622 | KNN Loss: 2.4050540924072266 | CLS Loss: 0.024537308141589165
Epoch 185 / 200 | iteration 170 / 171 | Total Loss: 2.407379150390625 | KNN Loss: 2.4010794162750244 | CLS Loss: 0.006299692206084728
Epoch: 185, Loss: 2.4127, Train: 0.9964, Valid: 0.9852, Best: 0.9874
Epoch 186 / 200 | iteration 0 / 171 | Total Loss: 2.3946070671081543 | KNN Loss: 2.3843612670898438 | CLS Loss: 0.0102457758

Epoch: 188, Loss: 2.4163, Train: 0.9968, Valid: 0.9856, Best: 0.9874
Epoch 189 / 200 | iteration 0 / 171 | Total Loss: 2.4022114276885986 | KNN Loss: 2.39015531539917 | CLS Loss: 0.01205622497946024
Epoch 189 / 200 | iteration 10 / 171 | Total Loss: 2.4021103382110596 | KNN Loss: 2.3974359035491943 | CLS Loss: 0.004674519412219524
Epoch 189 / 200 | iteration 20 / 171 | Total Loss: 2.4069371223449707 | KNN Loss: 2.3938426971435547 | CLS Loss: 0.013094501569867134
Epoch 189 / 200 | iteration 30 / 171 | Total Loss: 2.3911221027374268 | KNN Loss: 2.3857240676879883 | CLS Loss: 0.0053981030359864235
Epoch 189 / 200 | iteration 40 / 171 | Total Loss: 2.377262592315674 | KNN Loss: 2.370898723602295 | CLS Loss: 0.006363935302942991
Epoch 189 / 200 | iteration 50 / 171 | Total Loss: 2.475409984588623 | KNN Loss: 2.4523887634277344 | CLS Loss: 0.02302112616598606
Epoch 189 / 200 | iteration 60 / 171 | Total Loss: 2.4302632808685303 | KNN Loss: 2.4257004261016846 | CLS Loss: 0.00456285011023283
E

Epoch 192 / 200 | iteration 60 / 171 | Total Loss: 2.445564031600952 | KNN Loss: 2.4437737464904785 | CLS Loss: 0.0017902180552482605
Epoch 192 / 200 | iteration 70 / 171 | Total Loss: 2.4203107357025146 | KNN Loss: 2.4160983562469482 | CLS Loss: 0.004212354775518179
Epoch 192 / 200 | iteration 80 / 171 | Total Loss: 2.4323344230651855 | KNN Loss: 2.4142065048217773 | CLS Loss: 0.01812788099050522
Epoch 192 / 200 | iteration 90 / 171 | Total Loss: 2.41702938079834 | KNN Loss: 2.4050469398498535 | CLS Loss: 0.01198238879442215
Epoch 192 / 200 | iteration 100 / 171 | Total Loss: 2.3841145038604736 | KNN Loss: 2.379514694213867 | CLS Loss: 0.004599805921316147
Epoch 192 / 200 | iteration 110 / 171 | Total Loss: 2.4017508029937744 | KNN Loss: 2.399730920791626 | CLS Loss: 0.0020198074635118246
Epoch 192 / 200 | iteration 120 / 171 | Total Loss: 2.4248809814453125 | KNN Loss: 2.407672643661499 | CLS Loss: 0.01720832660794258
Epoch 192 / 200 | iteration 130 / 171 | Total Loss: 2.423055887222

Epoch 195 / 200 | iteration 120 / 171 | Total Loss: 2.3954384326934814 | KNN Loss: 2.3866419792175293 | CLS Loss: 0.00879654660820961
Epoch 195 / 200 | iteration 130 / 171 | Total Loss: 2.3906569480895996 | KNN Loss: 2.3863706588745117 | CLS Loss: 0.0042863208800554276
Epoch 195 / 200 | iteration 140 / 171 | Total Loss: 2.378544330596924 | KNN Loss: 2.371440887451172 | CLS Loss: 0.007103500887751579
Epoch 195 / 200 | iteration 150 / 171 | Total Loss: 2.413036346435547 | KNN Loss: 2.37988018989563 | CLS Loss: 0.03315627574920654
Epoch 195 / 200 | iteration 160 / 171 | Total Loss: 2.413043975830078 | KNN Loss: 2.409041166305542 | CLS Loss: 0.004002727568149567
Epoch 195 / 200 | iteration 170 / 171 | Total Loss: 2.436136245727539 | KNN Loss: 2.4286253452301025 | CLS Loss: 0.007510796654969454
Epoch: 195, Loss: 2.4133, Train: 0.9975, Valid: 0.9863, Best: 0.9874
Epoch 196 / 200 | iteration 0 / 171 | Total Loss: 2.405136823654175 | KNN Loss: 2.396449089050293 | CLS Loss: 0.008687762543559074

Epoch: 198, Loss: 2.4146, Train: 0.9965, Valid: 0.9855, Best: 0.9874
Epoch 199 / 200 | iteration 0 / 171 | Total Loss: 2.458559513092041 | KNN Loss: 2.4451005458831787 | CLS Loss: 0.013458899222314358
Epoch 199 / 200 | iteration 10 / 171 | Total Loss: 2.393702745437622 | KNN Loss: 2.3843774795532227 | CLS Loss: 0.009325203485786915
Epoch 199 / 200 | iteration 20 / 171 | Total Loss: 2.411714792251587 | KNN Loss: 2.408041477203369 | CLS Loss: 0.0036733821034431458
Epoch 199 / 200 | iteration 30 / 171 | Total Loss: 2.4443836212158203 | KNN Loss: 2.425159454345703 | CLS Loss: 0.01922428421676159
Epoch 199 / 200 | iteration 40 / 171 | Total Loss: 2.3968935012817383 | KNN Loss: 2.3940272331237793 | CLS Loss: 0.002866328228265047
Epoch 199 / 200 | iteration 50 / 171 | Total Loss: 2.4277451038360596 | KNN Loss: 2.412027359008789 | CLS Loss: 0.01571783795952797
Epoch 199 / 200 | iteration 60 / 171 | Total Loss: 2.4099137783050537 | KNN Loss: 2.4005932807922363 | CLS Loss: 0.009320582263171673
E

In [8]:
test(model, test_data_iter, device)

tensor(0.9856, device='cuda:0')

In [9]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

In [23]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyboardInterrupt: 

In [25]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [26]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.8591658672513819


In [27]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [28]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [29]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [30]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [31]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [32]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [33]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
layer 9: 0.0
layer 10: 0.0
Epoch: 00 | Batch: 000 / 037 | Total loss: 3.313 | Reg loss: 0.014 | Tree loss: 3.313 | Accuracy: 0.033203 | 7.816 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 01 | Batch: 000 / 037 | Total loss: 3.305 | Reg loss: 0.005 | Tree loss: 3.305 | Accuracy: 0.060547 | 5.372 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
l

Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 18 | Batch: 000 / 037 | Total loss: 3.137 | Reg loss: 0.015 | Tree loss: 3.137 | Accuracy: 0.289062 | 5.258 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 19 | Batch: 000 / 037 | Total loss: 3.138 | Reg loss: 0.015 | Tree loss: 3.138 | Accuracy: 0.269531 | 5.261 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.984042

layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 35 | Batch: 000 / 037 | Total loss: 3.127 | Reg loss: 0.018 | Tree loss: 3.127 | Accuracy: 0.255859 | 5.227 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895
Epoch: 36 | Batch: 000 / 037 | Total loss: 3.079 | Reg loss: 0.018 | Tree loss: 3.079 | Accuracy: 0.296875 | 5.226 sec/iter
Average sparseness: 0.9840425531914895
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
layer 5: 0.9840425531914894
layer 6: 0.9840425531914895
layer 7: 0.9840425531914895
layer 8: 0.9840425531914895
layer 9: 0.9840425531914895
layer 10: 0.9840425531914895

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")